---
# Cairo University Faculty of Engineering
## Machine Learning
## Assignment 3 - Decision trees from scratch

---

**Name** : Ibrahim Mohamed Ibrahim

### Data Set

In [58]:
import numpy as np
import pandas as pd
import time

from collections import Counter
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [59]:
## load data
cardio_df = pd.read_csv('./data/cardio_train.csv' , sep= ';', index_col=0)
cardio_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 70000 entries, 0 to 99999
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   age          70000 non-null  int64  
 1   gender       70000 non-null  int64  
 2   height       70000 non-null  int64  
 3   weight       70000 non-null  float64
 4   ap_hi        70000 non-null  int64  
 5   ap_lo        70000 non-null  int64  
 6   cholesterol  70000 non-null  int64  
 7   gluc         70000 non-null  int64  
 8   smoke        70000 non-null  int64  
 9   alco         70000 non-null  int64  
 10  active       70000 non-null  int64  
 11  cardio       70000 non-null  int64  
dtypes: float64(1), int64(11)
memory usage: 6.9 MB


In [60]:
## view the first 5 element of the data
cardio_df.head()

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
id,,,,,,,,,,,,
0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,17474,1,156,56.0,100,60,1,1,0,0,0,0


### Data Exploration

In [61]:
## convert age to years and round it
cardio_df['age'] = round(cardio_df['age'] / 365.25)
cardio_df.head(5)

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
id,,,,,,,,,,,,
0,50.0,2,168,62.0,110,80,1,1,0,0,1,0
1,55.0,1,156,85.0,140,90,3,1,0,0,1,1
2,52.0,1,165,64.0,130,70,3,1,0,0,0,1
3,48.0,2,169,82.0,150,100,1,1,0,0,1,1
4,48.0,1,156,56.0,100,60,1,1,0,0,0,0


In [62]:
## Get dataset statistics
cardio_df.describe().T

,count,mean,std,min,25%,50%,75%,max
age,70000.0,53.303157,6.760171,30.0,48.0,54.0,58.0,65.0
gender,70000.0,1.349571,0.476838,1.0,1.0,1.0,2.0,2.0
height,70000.0,164.359229,8.210126,55.0,159.0,165.0,170.0,250.0
weight,70000.0,74.205690,14.395757,10.0,65.0,72.0,82.0,200.0
ap_hi,70000.0,128.817286,154.011419,-150.0,120.0,120.0,140.0,16020.0
ap_lo,70000.0,96.630414,188.472530,-70.0,80.0,80.0,90.0,11000.0
cholesterol,70000.0,1.366871,0.680250,1.0,1.0,1.0,2.0,3.0
gluc,70000.0,1.226457,0.572270,1.0,1.0,1.0,1.0,3.0
smoke,70000.0,0.088129,0.283484,0.0,0.0,0.0,0.0,1.0
alco,70000.0,0.053771,0.225568,0.0,0.0,0.0,0.0,1.0


### My Decision Tree Class

In [71]:
class DecisionTree:
    def __init__(self, max_depth):
        self.max_depth = max_depth
        self.tree = {}

    def _calculate_entropy(self, y):
        _, counts = np.unique(y, return_counts=True)
        probs = counts / np.sum(counts)
        entropy = -np.sum(probs * np.log2(probs))

        return entropy

    def _calculate_information_gain(self, X, y, feature_idx, thresholds):
        n_samples = len(y)
        parent_entropy = self._calculate_entropy(y)

        child_entropy = 0
        for threshold in thresholds:
            idx = X[:, feature_idx] == threshold
            child_entropy += np.sum(idx) / n_samples * self._calculate_entropy(y[idx])

        gain = parent_entropy - child_entropy

        return gain

    def _get_best_feature(self, X, y):
        n_samples, n_features = X.shape
        best_feature_idx = None
        best_thresholds = None
        max_info_gain = -1

        for feature_idx in range(n_features):
            feature_values = X[:, feature_idx]
            thresholds = np.unique(feature_values)

            info_gain = self._calculate_information_gain(X, y, feature_idx, thresholds)

            if info_gain > max_info_gain:
                max_info_gain = info_gain
                best_feature_idx = feature_idx
                best_thresholds = thresholds

        return best_feature_idx, best_thresholds

    # Function to use when we reach the max depth
    def _most_common_label(self, y):
        unique, counts = np.unique(y, return_counts=True)
        index = np.argmax(counts)
        label = unique[index]
        return label


    def _build_tree(self, X, y, depth=0):
        n_samples, n_features = X.shape
        n_classes = len(np.unique(y))

        # Stopping criteria
        if depth >= self.max_depth or n_classes == 1:
            return self._most_common_label(y)

        # Split the data based on the best feature
        best_feature_idx, thresholds = self._get_best_feature(X, y)

        sub_trees = {}
        for threshold in thresholds:
            idx = X[:, best_feature_idx] == threshold
            sub_tree = self._build_tree(X[idx], y[idx], depth+1)
            sub_trees[threshold] = sub_tree

        # Node
        node = {'feature_idx': best_feature_idx,
                'thresholds': thresholds,
                'sub_trees': sub_trees}

        return node


    def fit(self, X, y):
        self.tree = self._build_tree(X, y)

    # Loop throght the tree nodes
    def _tree_traversal(self, x, node):
        if isinstance(node, np.int64):
            return node

        feature_idx = node['feature_idx']
        thresholds = node['thresholds']
        sub_trees = node['sub_trees']

        for threshold in thresholds:
            if x[feature_idx] == threshold:
                return self._tree_traversal(x, sub_trees[threshold])

        return self._most_common_label(y)

    def predict(self, X):
        return np.array([self._tree_traversal(x, self.tree) for x in X])

    def score(self, X, y):
        y_pred = self.predict(X)
        return np.mean(y == y_pred)

In [64]:
cont_columns = ['age', 'height', 'weight', 'ap_hi', 'ap_lo']

for col in cont_columns:
    threshold = cardio_df[col].mean()

    cardio_df['updated_{}'.format(col)] = cardio_df[col].apply(lambda x: 0 if x < threshold else 1)
    cardio_df.drop(col, axis=1, inplace=True)

In [65]:
cardio_df

,gender,cholesterol,gluc,smoke,alco,active,cardio,updated_age,updated_height,updated_weight,updated_ap_hi,updated_ap_lo
id,,,,,,,,,,,,
0,2,1,1,0,0,1,0,0,1,0,0,0
1,1,3,1,0,0,1,1,1,0,1,1,0
2,1,3,1,0,0,0,1,0,1,0,1,0
3,2,1,1,0,0,1,1,0,1,1,1,1
4,1,1,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
99993,2,1,1,1,0,1,0,0,1,1,0,0
99995,1,2,2,0,0,1,1,1,0,1,1,0
99996,2,3,1,0,1,0,1,0,1,1,1,0


In [66]:
## Split the dataset into features and target variable
X = cardio_df.drop(['cardio'], axis=1).values
y = cardio_df['cardio'].values



## Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [67]:
##  Create an instance of Sklearn Decision Tree with max_depth 5
skl_tree = DecisionTreeClassifier(criterion= 'entropy', max_depth= 5)

## Fitting decision tree model to data & calculating the time it takes
skl_train_time_start = time.time()
skl_tree.fit(X_train,y_train)
skl_train_time_end = time.time()

## Getting prediction from sklearn tree and calculate the time it takes
skl_pred_time_start = time.time()
skl_tree_y_pred = skl_tree.predict(X_test)
skl_pred_time_end = time.time()

# Calculate accuracy score of Sklearn tree predictions
skl_acc = accuracy_score(skl_tree_y_pred,y_test) * 100

In [73]:
# Create an instance of MyDecisionTree class with max_depth 5
my_tree = DecisionTree(max_depth=5)

## Fitting decision tree model to data & calculating the time it takes
my_train_time_start = time.time()
my_tree.fit(X_train, y_train)
my_train_time_end = time.time()


## Getting prediction from  My tree and calculate the time it takes
my_pred_time_start = time.time()
my_tree_y_pred = my_tree.predict(X_test)
my_pred_time_end = time.time()

## Calculate accuracy score of my tree predictions
my_acc = accuracy_score(y_test, my_tree_y_pred) * 100

In [74]:
## Calculate Sklearn tree time
skl_train_time = skl_train_time_end - skl_train_time_start
skl_pred_time = skl_pred_time_end - skl_pred_time_start

## Calculate my tree time
my_train_time = my_train_time_end - my_train_time_start
my_pred_time = my_pred_time_end  - my_pred_time_start

In [75]:
print(f"My Tree training time: {my_train_time}")
print(f"Scikit-Learn training time: {skl_train_time}")
print(50*'-')
print(f"My Tree prediction time: {my_pred_time}")
print(f"Scikit-Learn prediction time: {skl_pred_time}")
print(50*'-')
print(f"My accuracy: {my_acc}")
print(f"Scikit-Learn accuracy: {skl_acc}")

My Tree training time: 0.23337769508361816
Scikit-Learn training time: 0.035902976989746094
--------------------------------------------------
My Tree prediction time: 0.048867225646972656
Scikit-Learn prediction time: 0.0009982585906982422
--------------------------------------------------
My accuracy: 73.12857142857143
Scikit-Learn accuracy: 73.13571428571429
